In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")

driver = webdriver.Chrome(options=options)

df = pd.DataFrame(columns = ['Field', 'System Usage (%)', 'Org Usage (%)', 'Resource'])
# df = pd.DataFrame(columns = ['Field', 'System Usage (%)', 'Org Usage (%)', 'Resource', 'Group'])

# Open the URL
url = 'https://ddwiki.reso.org/display/DDW17/Property+Resource'
driver.get(url)

# identify elements with tagname <a>
links = driver.find_elements(By.TAG_NAME, "a")

all_href = []

# traverse list
for lnk in links:
   # get_attribute() to get all href
   all_href.append(lnk.get_attribute('href'))

In [ ]:
split_links = [i.split('?', 1)[0] for i in all_href]
field_links = [j for j in split_links if '+Field' in j]

# print(field_links) 

In [14]:
for lnk in field_links:
    page = requests.get(lnk)
    soup = BeautifulSoup(page.text, 'html')
    inner_cell_elements = soup.find_all('div', class_='innerCell')

    field_name = soup.find('title').text.strip().split(' ')[0]

    crumbs = soup.find_all('ol', id='breadcrumbs')

    for a in crumbs:
        a_tag = a.find_all('a')
        a_text = [txt.text.strip() for txt in a_tag]

    if len(a_text) < 3:
        a_text.insert(3, 'null')

    for li in inner_cell_elements[6:7]:
        li_tag = li.find_all('li')
        li_text = [data.text.strip() for data in li_tag[0:2]]

    df.loc[len(df.index)] = [field_name, li_text[0], li_text[1], a_text[2]]

print(df)

                      Field         System Usage (%)  \
0           PropertySubType   88% of Systems (14/16)   
1              PropertyType  100% of Systems (16/16)   
2       UniversalPropertyId    12% of Systems (2/16)   
3    UniversalPropertySubId     6% of Systems (1/16)   
4              BusinessName   69% of Systems (11/16)   
..                      ...                      ...   
612         PowerProduction    <1% of Systems (0/16)   
613     PowerProductionType    31% of Systems (5/16)   
614                   Sewer   88% of Systems (14/16)   
615               Utilities   88% of Systems (14/16)   
616             WaterSource   81% of Systems (13/16)   

                      Org Usage (%)           Resource  
0    92% of Organizations (452/491)  Property Resource  
1    98% of Organizations (480/491)  Property Resource  
2      <1% of Organizations (2/491)  Property Resource  
3      <1% of Organizations (1/491)  Property Resource  
4    28% of Organizations (137/491)  Prope

In [21]:
df['Resource'].replace(' Resource','', regex=True, inplace=True)
df.replace({'(?<=%).*': ''}, regex=True, inplace=True)
df = df[~df['System Usage (%)'].str.contains("[a-zA-Z]").fillna(False)]

df.to_csv('../RESO Scraper/reso_field_usage.csv')